In [1]:
import numpy as np
from scipy.integrate import trapz

#PYTHON MODULES
from spectral_sirens.utils.constants import *
from spectral_sirens.cosmology import gwcosmo
from spectral_sirens.gw_population import gwpop
from spectral_sirens.utils import gwutils
from spectral_sirens.utils import utils
from spectral_sirens.detectors import sensitivity_curves as sc

In [2]:
"""Astropy"""
import astropy.units as astrou
from astropy.cosmology import Planck18 as cosmo
from astropy.cosmology import z_at_value


def z_dl(dl):
    if dl<=1.0e-2:
        zdl = 0.
    elif dl<=1.0e12:
        zdl = z_at_value(cosmo.luminosity_distance,dl*astrou.kpc,zmax = 1.0e5)
    else:
        zdl = z_at_value(cosmo.luminosity_distance,1e12*astrou.kpc,zmax = 1.0e5)
    return zdl
vz_dl = np.vectorize(z_dl)

def z_dls(dl): #dl in kpc
    if dl < 1:
        zs = z_at_value(cosmo.luminosity_distance,1.*astrou.kpc,zmax = 1.0e5)
    elif dl < 1e11:
        zs = z_at_value(cosmo.luminosity_distance,dl*astrou.kpc,zmax = 1.0e5)
    else:
        zs = z_at_value(cosmo.luminosity_distance,1e11*astrou.kpc,zmax = 1.0e5)
    return zs
vz_dls = np.vectorize(z_dls)

In [3]:

def dhor(mass_1z,mass_2z,fmin_gw,Tobs,detectorSn,fmin_detect,fmax_detect,based,snr_th):
    # Mass in Msun at *detector* frame
    #f in *detector* frame
    #Tobs in *detector* frame
    dL = 1.0e-3 # Mpc
    return gwutils.vsnr_from_psd(mass_1z,mass_2z,dL,fmin_gw,Tobs,detectorSn,fmin_detect,fmax_detect,based) / snr_th #in KPC


""" Maximum Horizon Distance """
def dhor_max(mass_1z,mass_2z,fmin_gw,Tobs,detectorSn,fmin_detect,fmax_detect,based,snr_th):
    # Mass in Msun at *detector* frame
    dls = np.zeros(len(mass_1z))
    d = 1.0e-3 # Mpc
    for i in range(0,len(mass_1z)):
        fmax = gwutils.f_PhenomA(3,mass_1z[i],mass_2z[i])

        dli = np.zeros(len(fmin_gw))
        for j in range(0,len(fmin_gw)):
            if fmin_gw[j] < fmax:
                dli[j] = dhor(mass_1z[i],mass_2z[i],fmin_gw[j],Tobs,detectorSn,fmin_detect,fmax_detect,based,snr_th) #in KPC
                if dli[j] < 1.:
                    dli[j] = 1.
                elif dli[j] > 1.0e11: #Avoids divergences, should improve this!
                    dli[j] = 1.
            else:
                dli[j] = 1.
        dls[i] = max(dli)
    return dls
vdhor_max = np.vectorize(dhor_max)

def zhor_max(mass_1z,mass_2z,fmin_gw,Tobs,detectorSn,fmin_detect,fmax_detect,based,snr_th):
    #z_at_dl_approx(dl,H0,Om0,zmin=1e-3,zmax=50)
    return vz_dls(dhor_max(mass_1z,mass_2z,fmin_gw,Tobs,detectorSn,fmin_detect,fmax_detect,based,snr_th))

In [4]:
#Directories
import os 
PATH = os.getcwd()
dir_out = 'data_injections/'

#PLOTS
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
fontSz = 15
fontsz = 13
fontssz = 11

new_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf']

In [23]:
Sn_O3, fmin_O3, fmax_O3 = sc.detector_psd('O3')
Sn_O4, fmin_O4, fmax_O4 = sc.detector_psd('O4')
Sn_Aplus, fmin_Aplus, fmax_Aplus = sc.detector_psd('A+')
Sn_Asharp, fmin_Asharp, fmax_Asharp = sc.detector_psd('A#')
Sn_CE, fmin_CE, fmax_CE = sc.detector_psd('CE-40')
based = 'ground'
snr_th = 8.

In [6]:
from scipy.interpolate import interp1d

""" Horizon distance """
datazhorLISA = np.genfromtxt('gw_detector_horizon/zhor_max_lisa.txt')
massesLISA=datazhorLISA[:,0]
zhorLISA=datazhorLISA[:,1]
zhormaxLISA = interp1d(massesLISA,zhorLISA,bounds_error=False,fill_value=0.0)

datazhorLISAsky = np.genfromtxt('gw_detector_horizon/zhor_max_lisa_sky.txt')
massesLISAsky=datazhorLISAsky[:,0]
zhorLISAsky=datazhorLISAsky[:,1]
zhormaxLISAsky = interp1d(massesLISAsky,zhorLISAsky,bounds_error=False,fill_value=0.0)

datazhorO2 = np.genfromtxt('gw_detector_horizon/zhor_max_o2.txt')
massesO2=datazhorO2[:,0]
zhorO2=datazhorO2[:,1]
zhormaxO2 = interp1d(massesO2,zhorO2,bounds_error=False,fill_value=0.0)

datazhorO3 = np.genfromtxt('gw_detector_horizon/zhor_max_o3.txt')
massesO3=datazhorO3[:,0]
zhorO3=datazhorO3[:,1]
zhormaxO3 = interp1d(massesO3,zhorO3,bounds_error=False,fill_value=0.0)

datazhorO4 = np.genfromtxt('gw_detector_horizon/zhor_max_o4.txt')
massesO4=datazhorO4[:,0]
zhorO4=datazhorO4[:,1]
zhormaxO4 = interp1d(massesO4,zhorO4,bounds_error=False,fill_value=0.0)

datazhorO5 = np.genfromtxt('gw_detector_horizon/zhor_max_o5.txt')
massesO5=datazhorO5[:,0]
zhorO5=datazhorO5[:,1]
zhormaxO5 = interp1d(massesO5,zhorO5,bounds_error=False,fill_value=0.0)

datazhorvoyager = np.genfromtxt('gw_detector_horizon/zhor_max_voyager.txt')
massesvoyager=datazhorvoyager[:,0]
zhorvoyager=datazhorvoyager[:,1]
zhormaxvoyager = interp1d(massesvoyager,zhorvoyager,bounds_error=False,fill_value=0.0)

datazhorLIGOeh = np.genfromtxt('gw_detector_horizon/zhor_max_ligo_eh.txt')
massesLIGOeh=datazhorLIGOeh[:,0]
zhorLIGOeh=datazhorLIGOeh[:,1]
zhormaxLIGOeh = interp1d(massesLIGOeh,zhorLIGOeh,bounds_error=False,fill_value=0.0)

datazhorALIGO = np.genfromtxt('gw_detector_horizon/zhor_max_aligo.txt')
massesALIGO=datazhorALIGO[:,0]
zhorALIGO=datazhorALIGO[:,1]
zhormaxALIGO = interp1d(massesALIGO,zhorALIGO,bounds_error=False,fill_value=0.0)

datazhorAplus = np.genfromtxt('gw_detector_horizon/zhor_max_aplus.txt')
massesAplus=datazhorAplus[:,0]
zhorAplus=datazhorAplus[:,1]
zhormaxAplus = interp1d(massesAplus,zhorAplus,bounds_error=False,fill_value=0.0)

datazhorET = np.genfromtxt('gw_detector_horizon/zhor_max_et.txt')
massesET=datazhorET[:,0]
zhorET=datazhorET[:,1]
zhormaxET = interp1d(massesET,zhorET,bounds_error=False,fill_value=0.0)

datazhorCE = np.genfromtxt('gw_detector_horizon/zhor_max_ce.txt')
massesCE=datazhorCE[:,0]
zhorCE=datazhorCE[:,1]
zhormaxCE = interp1d(massesCE,zhorCE,bounds_error=False,fill_value=0.0)

In [28]:
nfs = 100
fs_O3=np.logspace(np.log10(fmin_O3),np.log10(fmax_O3),nfs)
fs_O4=np.logspace(np.log10(fmin_O4),np.log10(fmax_O4),nfs)
fs_Aplus=np.logspace(np.log10(fmin_Aplus),np.log10(fmax_Aplus),nfs)
fs_Asharp=np.logspace(np.log10(fmin_Asharp),np.log10(fmax_Asharp),nfs)
fs_CE=np.logspace(np.log10(fmin_CE),np.log10(fmax_CE),nfs)

nms = 500
masses = np.logspace(np.log10(0.5),5,nms)
masses_ext = np.logspace(np.log10(0.5),7.,nms)
Tobs = 1.0

zhor_O3 = zhor_max(masses,masses,fs_O3,Tobs,Sn_O3,fmin_O3,fmax_O3,based,snr_th)
np.savetxt('gw_detector_horizon/zhor_max_o3.txt',np.column_stack((2.*masses/(1.+zhor_O3),zhor_O3)),delimiter = ' ')

zhor_O4 = zhor_max(masses,masses,fs_O4,Tobs,Sn_O4,fmin_O4,fmax_O4,based,snr_th)
np.savetxt('gw_detector_horizon/zhor_max_o4.txt',np.column_stack((2.*masses/(1.+zhor_O4),zhor_O4)),delimiter = ' ')

zhor_Aplus = zhor_max(masses,masses,fs_Aplus,Tobs,Sn_Aplus,fmin_Aplus,fmax_Aplus,based,snr_th)
np.savetxt('gw_detector_horizon/zhor_max_aplus.txt',np.column_stack((2.*masses/(1.+zhor_Aplus),zhor_Aplus)),delimiter = ' ')

zhor_Asharp = zhor_max(masses,masses,fs_Asharp,Tobs,Sn_Asharp,fmin_Asharp,fmax_Asharp,based,snr_th)
np.savetxt('gw_detector_horizon/zhor_max_asharp.txt',np.column_stack((2.*masses/(1.+zhor_Asharp),zhor_Asharp)),delimiter = ' ')

zhor_CE = zhor_max(masses,masses,fs_CE,Tobs,Sn_CE,fmin_CE,fmax_CE,based,snr_th)
np.savetxt('gw_detector_horizon/zhor_max_ce.txt',np.column_stack((2.*masses/(1.+zhor_CE),zhor_CE)),delimiter = ' ')